In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.inspection import DecisionBoundaryDisplay

from scipy import stats
from pca import pca
from IPython.display import display

from src.visualization import feature_importances_plot

import sys
import warnings
    
# warnings -> to silence warnings

warnings.filterwarnings("ignore")
np.set_printoptions(precision=5, suppress=True)


RANDOM_STATE = 42
N_JOBS = -1
class_names = ["Canis", "Dysg. Equisimilis", "Dysg. Dysgalactiae"]

map_target = {
    "Streptococcus canis": 0,
    "Streptococcus dysgalactiae subsp. equisimilis": 1,
    "Streptococcus dysgalactiae subsp. dysgalactiae": 2
}

map_target_inv = {
    0: "Strept. canis",
    1: "Strept. dysg. equisimilis",
    2: "Strept. dysg. dysgalactiae"
}

df_46 = pd.read_csv("data/Dati_Matemaldomics_46picchi.csv",
                delimiter=';', index_col='ID Strain')
df_306 = pd.read_csv("data/Dati_Matemaldomics_306picchi.csv",
                delimiter=';', index_col='ID Strain')

maldi_46 = df_46[df_46.columns[9:55]]
df_46['target'] = df_46["Putative Subspecies"].map(map_target)

maldi_306 = df_306[df_306.columns[9:315]]
df_306['target'] = df_306["Putative Subspecies"].map(map_target)

maldi_46.fillna(0, inplace=True)
maldi_46 = maldi_46.replace(',', '.', regex=True)
columns = maldi_46.columns
for column in columns:
    maldi_46[column] = maldi_46[column].astype(float)
display(maldi_46)

maldi_306.fillna(0, inplace=True)
maldi_306 = maldi_306.replace(',', '.', regex=True)
columns = maldi_306.columns
for column in columns:
    maldi_306[column] = maldi_306[column].astype(float)
display(maldi_306)

dataframes = [['maldi_46' , maldi_46 , df_46['target']],['maldi_306', maldi_306, df_306['target']]]
scaler = MinMaxScaler()
models = {'Logistic Regression' : LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs', max_iter=1000),
          'Decision Trees' : DecisionTreeClassifier(random_state=RANDOM_STATE),
          'K-nn' : KNeighborsClassifier(), 
          'Random Forest' : RandomForestClassifier(oob_score=False, n_jobs=N_JOBS, random_state=RANDOM_STATE)}
type_features = ["all_features", "pca"]
cv_scores = list()
ac_scores = list()
cv_scores_base = list()
ac_scores_base = list()
data_cv = list()
data_ac = list()
data_cv_base = list()
data_ac_base = list()

y = df_46['target']
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
x = 0
t = 0
for type_feature in type_features:
    cv_scores.append(list())
    ac_scores.append(list())
    cv_scores_base.append(list())
    ac_scores_base.append(list())
    d=0
    #for str_df, X in dataframes.items():
    for dataframe in dataframes:
        str_df = dataframe[0]
        X = dataframe[1]
        y = dataframe[2]
        cv_scores[t].append(list())
        ac_scores[t].append(list())
        cv_scores_base[t].append(list())
        ac_scores_base[t].append(list())
        s=0
        
        X_scaled = scaler.fit_transform(X)
        
        if type_feature == "pca":
            model_pca = PCA(n_components = 0.95)
            X_reduced = model_pca.fit_transform(X_scaled)
            print("Feature con PCA:", X_train.shape)
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y)
        
        data_cv.append(list())
        data_ac.append(list())
        data_cv_base.append("Base_"+str_df+"_"+type_feature)
        data_cv[x].append("Best_"+str_df+"_"+type_feature)
        data_ac[x].append("Best_"+str_df+"_"+type_feature)
        m=0
        for str_model, model in models.items():
            model.fit(X_train, y_train)
            cv_scores_base[t][d].append(cross_val_score(estimator=model, X=X_train, y=y_train,
                                                    scoring="accuracy", cv=skfold, n_jobs=N_JOBS, verbose=1))
            print(str_model+" Base"+
                "\nDataframe: "+str_df+
                "\nFeatures: "+type_feature)
            print(model.get_params())
            print(f"Mean CV accuracy: {cv_scores_base[t][d][m].mean():.4f} +/- {cv_scores_base[t][d][m].std():.4f}")
            print(cv_scores_base[t][d][m])

            '''
            y_pred = model.predict(X_test)
            ac_scores[t][d][s].append(model.score(X_test, y_test))
            report = classification_report(y_true=y_test, y_pred=y_pred)
            print(report)
            '''
            if str_model == 'Logistic Regression':
                model = LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs', max_iter=1000)
                params = {
                    "penalty": ["l2", "l1"],
                    "C": stats.loguniform(1e0, 1e2),
                    "class_weight": [None, "balanced"]
                }
            if str_model == 'Decision Trees':
                print('Altezza albero base'+str(model.get_depth()))
                model = DecisionTreeClassifier(random_state=RANDOM_STATE)
                path = model.cost_complexity_pruning_path(X=X_train, y=y_train)
                ccp_alphas, impurities = path.ccp_alphas, path.impurities
                
                params = {
                    'ccp_alpha': ccp_alphas,
                    'max_depth': [None,3,5,7,10,15],
                    'min_samples_leaf': [1,3,5,10,15,20],
                    'min_samples_split': [2,4,6,8,10,12,14,16,18,20],
                    'criterion': ['gini','entropy']
                }
            if str_model == 'K-nn':
                model = KNeighborsClassifier()
                params = {
                    'n_neighbors' : [5,7,9,11,13,15],
                    'weights' : ['uniform','distance'],
                    'metric' : ['minkowski','euclidean','manhattan']
                }
            if str_model == 'Random Forest':
                model = RandomForestClassifier(oob_score=True, n_jobs=N_JOBS, random_state=RANDOM_STATE)
                params = {
                    "n_estimators": [25, 50, 100, 200, 250, 500],
                    "criterion": ["gini", "entropy"],
                    "max_depth": [None, 1, 2, 5, 10, 20],
                    "max_features": ["sqrt", "log2"],
                    "class_weight": [None, "balanced", "balanced_subsample"]
                }
            rs = RandomizedSearchCV(estimator=model, param_distributions=params,
                                scoring="accuracy", n_jobs=-1, cv=skfold, verbose=1)
            rs.fit(X_train, y_train)
            print(str_model+" Best \
                \nDataframe: "+str_df+
                "\nFeatures: "+type_feature)
            print(rs.best_params_)
            print('Random '+str_model+': '+str(rs.best_score_))
            parametri = rs.best_params_
            model.set_params(**parametri)
            print(model.get_params())
            model.fit(X_train, y_train)
            cv_scores[t][d].append(cross_val_score(estimator=model, X=X_train, y=y_train,
                                                    scoring="accuracy", cv=skfold, n_jobs=N_JOBS, verbose=1))
            print(f"Mean CV accuracy: {cv_scores[t][d][m].mean():.4f} +/- {cv_scores[t][d][m].std():.4f}")
            print(cv_scores[t][d][m])
            
            fig, ax = plt.subplots(figsize=(8, 5))
            plot_confusion_matrix(estimator=model, X=X_test, y_true=y_test,
                                cmap='Blues', display_labels=class_names, ax=ax)
            plt.title(str_model+" with best params")
            plt.tight_layout()
            plt.show()
            
            y_pred = model.predict(X_test)
            ac_scores[t][d].append(model.score(X_test, y_test))
            report = classification_report(y_true=y_test, y_pred=y_pred)
            print(report)

            if str_model != 'K-nn' and str_model != 'Random Forest':
                #Misura il learning curve del nuovo modello
                train_sizes = np.linspace(0.01, 1, 20)
                train_sizes, train_scores, valid_scores = learning_curve(estimator=model, X=X_train, y=y_train,
                                                                        train_sizes=train_sizes, cv=5,
                                                                        scoring="accuracy", n_jobs=-1, shuffle=True, random_state=42)

                train_mean = train_scores.mean(axis=1)
                valid_mean = valid_scores.mean(axis=1)

            #plt.style.use('seaborn')
            plt.figure(figsize=(8, 5))
            plt.plot(train_sizes, train_mean, label='Training error')
            plt.plot(train_sizes, valid_mean, label='Validation error')
            plt.ylabel('MSE', fontsize=14)
            plt.xlabel('Training set size', fontsize=14)
            plt.title('Learning curves for '+str_model+' with best parameter',
                    fontsize=18, y=1.03)
            plt.ylim(0, 2)
            plt.legend()
            plt.show()
            
            if str_model == 'Logistic Regression' and type_feature != 'pca':
                alphas = np.logspace(-5, 0, 100)
                coefs_ridge = []
                metrics_ridge = []
                coefs_lasso = []
                metrics_lasso = []
                    
                #Rifa ridge e lasso ma questa volta direttamente su le feature, senza polynomial features
                for alpha in alphas:
                    ridge = Ridge(alpha=alpha)
                    ridge.fit(X_train, y_train)
                    coefs_ridge.append(ridge.coef_)
                    metrics_ridge.append({"lambda": alpha, "r2_train": ridge.score(X_train, y_train), "r2_test": ridge.score(X_test, y_test)})
                    lasso = Lasso(alpha=alpha)
                    lasso.fit(X_train, y_train)
                    coefs_lasso.append(lasso.coef_)
                    metrics_lasso.append({"lambda": alpha, "r2_train": lasso.score(X_train, y_train), "r2_test": lasso.score(X_test, y_test)})
                    
                plt.plot(alphas, coefs_ridge)
                plt.xscale("log")
                plt.title("Ridge without Polynomial Features")
                plt.tight_layout()
                plt.grid()
                plt.show()

                plt.plot(alphas, coefs_lasso)
                plt.xscale("log")
                plt.title("Lasso without Polynomial Features")
                plt.tight_layout()
                plt.grid()
                plt.show()

                df_metrics_ridge = pd.DataFrame(metrics_ridge)
                df_metrics_ridge["difference"] = df_metrics_ridge["r2_train"] - df_metrics_ridge["r2_test"]

                df_metrics_lasso = pd.DataFrame(metrics_lasso)
                df_metrics_lasso["difference"] = df_metrics_lasso["r2_train"] - df_metrics_lasso["r2_test"]

                sns.lineplot(data=df_metrics_ridge, x="lambda", y="r2_train", label="train scores ridge")
                sns.lineplot(data=df_metrics_ridge, x="lambda", y="r2_test", label="test scores ridge")
                sns.lineplot(data=df_metrics_lasso, x="lambda", y="r2_train", label="train scores lasso")
                sns.lineplot(data=df_metrics_lasso, x="lambda", y="r2_test", label="test scores lasso")
                plt.xscale("log")
                plt.title("Score ridge/lasso without polynomial features")
                plt.grid()
                plt.tight_layout()
                plt.show()

                sns.lineplot(data=df_metrics_ridge, x="lambda", y="difference", label="difference scores ridge")
                sns.lineplot(data=df_metrics_lasso, x="lambda", y="difference", label="difference scores lasso")
                plt.xscale("log")
                plt.title("Difference between train and test score for ridge/lasso without polynomial features")
                plt.grid()
                plt.tight_layout()
                plt.show()
                
            if str_model == 'Decision Trees' and type_feature != 'pca':
                #Dizionario con importanza feature
                feature_importances = model.feature_importances_
                feature_index = X.columns
                myDict = dict(zip(feature_index, feature_importances))
                myDict = dict(sorted(myDict.items(), key=lambda item: item[1], reverse = False))

                #Plot delle 15 feature con più importanza
                series = pd.Series(data=myDict.values(), index=myDict.keys()).tail(10)
                series.plot(kind="barh", figsize=(8, 5), title=f"Feature importances for Decision Tree", legend=None)
                plt.tight_layout()
                plt.show()

                feature_names = feature_index

                plt.figure(figsize=(12, 12))
                plot_tree(decision_tree=model, 
                        feature_names=feature_names, 
                        class_names=class_names, 
                        filled=True, fontsize=8)
                plt.title("Decision Tree")
                plt.show()
            if str_model == 'K-nn' and type_feature != 'pca':
                knn_scores = []
                for k in range(1, X_train.shape[0], 2):
                    # model definition and training
                    knn =  KNeighborsClassifier(n_neighbors=k)
                    knn.fit(X=X_train, y=y_train)
                    # compute accuracy on test set
                    accuracy = knn.score(X_test, y_test)
                    # store the results on a list of dictionaries
                    metrics = {"# neighbors": k, "accuracy": accuracy}
                    knn_scores.append(metrics)

                # convert the list of dictionaries to pandas dataframe
                df_knn_scores = pd.DataFrame(data=knn_scores)    #molto facile eveloce da dizionario a dataframe 
                display(df_knn_scores)

                mask = df_knn_scores["accuracy"] == df_knn_scores["accuracy"].max()
                knn_k = df_knn_scores['accuracy'].idxmax()
                n = df_knn_scores['# neighbors'][knn_k]

                plt.figure(figsize=(7, 5))
                plt.title("KNN accuracy as function of the number of neighbors")
                sns.lineplot(x="# neighbors", y="accuracy", data=df_knn_scores)
                plt.grid(linestyle='-.', linewidth=0.5)
                plt.tight_layout()
                plt.show()
                display(df_knn_scores[mask])
                
            if str_model == 'Random Forest' and type_feature != 'pca':
                feature_importances_plot(model=model, labels=X.columns)
                print("Score Random Forest base: ", model.oob_score_)

                r2_rf = r2_score(y_test, y_pred)
                print('R2 score of random forest classifier on test set: {:.3f}'.format(r2_rf))

                rmse_rf = mean_squared_error(y_test, y_pred, squared=False)
                print('Mean squared error of random forest classifier on test set: {:.3f}'.format(rmse_rf))
                
            plt.figure(figsize=(6, 4))
            plt.title('CV '+str_model+' model scores')
            plt.plot(cv_scores_base[t][d][m], marker="o")
            plt.plot(cv_scores[t][d][m], marker="o")
            plt.xticks(ticks=range(1, 5))
            plt.xlabel("fold")
            plt.ylabel("accuracy")
            plt.legend(["Base model", "Best model"])
            plt.tight_layout()
            plt.grid()
            plt.show()
            data_cv[x].append(str(cv_scores[t][d][m].mean()))
            data_ac[x].append(str(ac_scores[t][d][m]))
            m += 1
        x += 1           
        d += 1
    t += 1

feature_index = ['Feature','Logistic Regression', 'Decision Tree', 'Knn', 'Random Forest']
df_ac_best = pd.DataFrame(data_ac, columns=feature_index)
df_ac_best.to_csv('accuracy_maldi.csv', index=False, mode='w')
df_cv_best = pd.DataFrame(data_cv, columns=feature_index)
df_cv_best.to_csv('cv_maldi.csv', index=False, mode='w')

    
print("AC SCORES BEST")
display(df_ac_best)
print("CV SCORES BEST")
display(df_cv_best)